In [57]:
import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

import math
import itertools

from datetime import datetime

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import time

In [43]:
from scipy import optimize

In [44]:
def inventory_cost_df(days, P_list, D_list, Q_list, S=240, C=3, IC=10, fast=False):
    
    ### params ###
    # 单位订货固定成本（shipment cost）：S = 240
    # 单位产品日均存货成本：C = 3
    # 单位产品日均缺货成本：IC = 10

    # 准备空列表
    day_i_list = [i for i in range(days)] # 总天数
    Stock_list = [None for i in range(days)]
    Ins_list = [None for i in range(days)]
    Cost_pq_list = [None for i in range(days)]
    Cost_ship_list = [None for i in range(days)]
    Cost_stock_list = [None for i in range(days)]
    Cost_ins_list = [None for i in range(days)]
    Cost_list = [None for i in range(days)]
    
    ### 按日处理 ###
    for day_i in range(days):
        
        # 昨日库存：
        if day_i == 0:
            l_stock = 0
        else:
            l_stock = Stock_list[day_i-1]
        # 今日价格：
        p = P_list[day_i]
        # 今日订货量：
        q = Q_list[day_i]
        # 今日需求量：
        d = D_list[day_i]
        # 更新今日库存 = 昨日库存 + 今日购进 - 今日使用量
        stock = l_stock + q - d # 今日结束时的库存（负数表示缺货）
        Stock_list[day_i] = stock # 记录
        # 今日缺货量：
        if Stock_list[day_i] < 0:
            ins = abs(Stock_list[day_i]) # 缺货量（正数）
        else:
            ins = 0
        Ins_list[day_i] = ins # 记录
        
        ### 今日成本计算 ###
        # 购进产品总价值
        cost_pq = q * p 
        # 运输成本 ，如果进货（q > 0），则付出运输成本 S，否则为 0
        if q > 0:
            cost_ship = S 
        else:
            cost_ship = 0
        # 存储成本，假设日内需求均匀发生，带来半数的等效库存成本
        cost_stock = C * (max(stock, 0) + min(d, max(q + l_stock, 0))/2) 
        # 缺货成本 = IC * 缺货量（正数）
        cost_ins = IC * ins 
        # 总成本
        cost = cost_pq + cost_ship + cost_stock + cost_ins 
        
        # 修改相应 list 中该日的数值
        Cost_pq_list[day_i] = cost_pq
        Cost_ship_list[day_i] = cost_ship
        Cost_stock_list[day_i] = cost_stock
        Cost_ins_list[day_i] = cost_ins
        Cost_list[day_i] = cost
        
    ### 存入 pandas df ###
    if fast == True:
        data = Cost_list
        df = pd.DataFrame(data=data, columns=['总成本'])
        df['累计总成本'] = df['总成本'].cumsum()
    else:
        data = zip(day_i_list, P_list, D_list, Q_list, Stock_list, Ins_list, Cost_list, Cost_pq_list, Cost_ship_list, Cost_stock_list, Cost_ins_list)
        df = pd.DataFrame(data=data, columns=['日期', '价格', '需求量', '订货量', '库存量', '缺货量', '总成本', '购进货物价值', '运输成本', '库存成本', '缺货成本'])
        df['累计需求量'] = df['需求量'].cumsum()
        df['累计订货量'] = df['订货量'].cumsum()
        df['累计总成本'] = df['总成本'].cumsum()
        df['累计运输成本'] = df['运输成本'].cumsum()
        df['累计库存成本'] = df['库存成本'].cumsum()
        df['累计缺货成本'] = df['缺货成本'].cumsum()
    
    #### return ###
    return df

In [45]:
# 总天数：
days = 60
day_i_list = [i for i in range(days)]
# 单位订货固定成本（shipment cost）：
S = 240.0
# 单位产品日均存货成本：
C = 3.0
# 单位产品日均缺货成本：
IC = 10.0

In [46]:
def solution_min_batchcost(days, P_list, D_list):

    # 空列表
    Q_list_selected = [0 for i in range(days)]
    Stock_list = [None for i in range(days)]
    
    # 逐日处理
    for day_i in range(days):

        # 如果昨日已经购进过，不再购进
        if Q_list_selected[day_i-1] != 0:
            best_Q = 0
        else:
            # reset record
            lowest_record = 9999999999

            ### 进货目标为：使本次进货的单位成本最低 ###

            # 仍需购进的总数：remain_num, 期初库存：begin_stock
            if day_i == 0:
                remain_num = sum(D_list)
                begin_stock = 0
            else:
                remain_num = sum(D_list[day_i:])
                begin_stock = Stock_list[day_i-1]

            ### 假设当日订购 try_num 个，后续都是 0，判断单位成本（try_num: 1～200）###
            for try_num in range(1, min(remain_num + 1, 201)):
                # Planned_stock_list
                Planned_stock_list = [None for i in range(days)]
                if day_i == 0:
                    Planned_Q_list = [0 for i in range(0, days)]
                    Planned_Q_list[day_i] = try_num
                else:
                    Planned_Q_list = Q_list_selected[0:day_i] + [0 for i in range(day_i, days)]
                    Planned_Q_list[day_i] = try_num
                # cycle_end
                for cycle_end in range(day_i, days):
                    # cycle_end 这一天的日末库存
                    if day_i == 0 or cycle_end == day_i:
                        Planned_stock_list[cycle_end] = try_num - D_list[cycle_end]
                    else:
                        Planned_stock_list[cycle_end] = Planned_stock_list[cycle_end-1] - D_list[cycle_end]
                    # 如果库存 <= 上一次订购后的库存量，则该周期结束，得到 cycle_end 的值
                    if Planned_stock_list[cycle_end] <= (begin_stock+try_num)/2:
                        break

                ### 本周期的总成本 ###
                # 起始数：day_i-1 的总成本
                if day_i == 0:
                    cost_begin = 0
                else:
                    cost_begin = inventory_cost_df(days, P_list, D_list, Planned_Q_list, fast=True)['累计总成本'].iloc[day_i-1]
                # 截止数：cycle_end 的总成本
                cost_end = inventory_cost_df(days, P_list, D_list, Planned_Q_list, fast=True)['累计总成本'].iloc[cycle_end]
                # 订购 try_num 个货物的 unit_cost
                unit_cost_this_cycle = (cost_end - cost_begin) / try_num  
                # 寻找新的 lowest_record
                if unit_cost_this_cycle < lowest_record:
                    lowest_record = unit_cost_this_cycle
                    best_Q = try_num

            # 如果不进货，则需要考虑由于今日不进货所导致的缺货成本 (D_list[day_i] - begin_stock) * IC
            target_unit_cost = 19000 / sum(D_list)
            inaction_unit_cost = (D_list[day_i] - begin_stock) * IC / 1
            # unit cost 过高，则选择不进货：既高于设定成本上限，又高于本日不进货的假设成本
            if lowest_record > target_unit_cost and lowest_record > inaction_unit_cost:
                best_Q = 0

        # 本日 Q 确定完毕，记录到 Q_list_selected
        Q_list_selected[day_i] = best_Q
        # 更新实际历史库存信息
        if day_i == 0:
            Stock_list[day_i] = Q_list_selected[0] - D_list[day_i]
        else:
            Stock_list[day_i] = Stock_list[day_i-1] + Q_list_selected[day_i] - D_list[day_i]

    # cost of Q_list_selected
    best_df = inventory_cost_df(days, P_list, D_list, Q_list_selected)
    return best_df

In [47]:
days = 60
P_list = [36.568, 36.683, 36.259, 37.763, 39.068, 36.532, 39.843, 40.769, 41.537, 40.724, 42.554, 45.006, 43.633, 47.1, 45.065, 47.923, 48.021, 47.656, 48.519, 45.427, 49.452, 46.887, 48.85, 49.831, 49.905, 49.104, 47.204, 48.517, 45.567, 46.332, 45.86, 45.569, 46.837, 43.411, 45.747, 43.281, 46.887, 46.113, 48.319, 49.988, 47.169, 50.355, 51.442, 49.871, 52.07, 51.174, 52.463, 52.186, 53.549, 52.53, 54.842, 55.305, 55.662, 55.447, 56.168, 57.047, 57.12, 56.782, 55.707, 57.207]
D_list = [21, 20, 23, 23, 39, 38, 26, 23, 22, 23, 25, 40, 31, 27, 23, 25, 26, 24, 43, 45, 28, 26, 24, 25, 29, 50, 45, 28, 24, 23, 23, 25, 50, 50, 29, 28, 29, 28, 29, 44, 14, 30, 31, 28, 29, 30, 59, 62, 37, 29, 30, 31, 31, 52, 53, 34, 28, 27, 28, 26]
P1 = P_list
D1 = D_list

In [48]:
our_Q_list = solution_min_batchcost(days, P1, D1)['订货量']
inventory_cost_df(days, P1, D1, our_Q_list)

,日期,价格,需求量,订货量,库存量,缺货量,总成本,购进货物价值,运输成本,库存成本,缺货成本,累计需求量,累计订货量,累计总成本,累计运输成本,累计库存成本,累计缺货成本
0,0,36.568,21,42,21,0,1870.356,1535.856,240,94.5,0,21,42,1870.356,240,94.5,0
1,1,36.683,20,0,1,0,33.000,0.000,0,33.0,0,41,42,1903.356,240,127.5,0
2,2,36.259,23,93,71,0,3859.587,3372.087,240,247.5,0,64,135,5762.943,480,375.0,0
3,3,37.763,23,0,48,0,178.500,0.000,0,178.5,0,87,135,5941.443,480,553.5,0
4,4,39.068,39,0,9,0,85.500,0.000,0,85.5,0,126,135,6026.943,480,639.0,0
5,5,36.532,38,85,56,0,3570.220,3105.220,240,225.0,0,164,220,9597.163,720,864.0,0
6,6,39.843,26,0,30,0,129.000,0.000,0,129.0,0,190,220,9726.163,720,993.0,0
7,7,40.769,23,0,7,0,55.500,0.000,0,55.5,0,213,220,9781.663,720,1048.5,0
8,8,41.537,22,51,36,0,2499.387,2118.387,240,141.0,0,235,271,12281.050,960,1189.5,0
9,9,40.724,23,0,13,0,73.500,0.000,0,73.5,0,258,271,12354.550,960,1263.0,0


## 局部优化 ##

In [49]:
def inventory_cost(days, P_list, D_list, Q_list, S=240, C=3, IC=10):
    
    # 准备空列表
    Stock_list = [None for i in range(days)]
    Cost_list = [None for i in range(days)]
    
    ### 按日处理 ###
    for day_i in range(days):
        
        # 昨日库存：
        if day_i == 0:
            l_stock = 0
        else:
            l_stock = Stock_list[day_i-1]
        # 今日价格：
        p = P_list[day_i]
        # 今日订货量：
        q = Q_list[day_i]
        # 今日需求量：
        d = D_list[day_i]
        # 更新今日库存 = 昨日库存 + 今日购进 - 今日使用量
        stock = l_stock + q - d # 今日结束时的库存（负数表示缺货）
        Stock_list[day_i] = stock # 记录
        # 今日缺货量：
        if Stock_list[day_i] < 0:
            ins = abs(Stock_list[day_i]) # 缺货量（正数）
        else:
            ins = 0

        ### 今日成本计算 ###
        # 购进产品总价值
        cost_pq = q * p 
        # 运输成本 ，如果进货（q > 0），则付出运输成本 S，否则为 0
        if q > 0:
            cost_ship = S 
        else:
            cost_ship = 0
        # 存储成本，假设日内需求均匀发生，带来半数的等效库存成本
        cost_stock = C * (max(stock, 0) + min(d, max(q + l_stock, 0))/2) 
        # 缺货成本 = IC * 缺货量（正数）
        cost_ins = IC * ins 
        # 总成本
        cost = cost_pq + cost_ship + cost_stock + cost_ins 
        
        # 修改相应 list 中该日的数值
        Cost_list[day_i] = cost
        
    return sum(Cost_list)

In [59]:
start = time.time()

func = lambda x: inventory_cost(days, P1, D1, x)
x0 = our_Q_list # 起始点
lw = [0] * 60
up = [200] * 60
our_Q_list_new = [int(i) for i in optimize.minimize(func, x0, bounds=list(zip(lw, up)))['x']]

inventory_cost_df(days, P1, D1, our_Q_list_new)

end = time.time()
run_time = end - start
print(run_time)

12.548205137252808


In [51]:
print([i for i in our_Q_list_new])

[41, 0, 84, 0, 0, 87, 0, 0, 44, 0, 64, 0, 81, 0, 0, 50, 0, 131, 0, 0, 0, 58, 0, 53, 0, 115, 0, 0, 63, 0, 0, 139, 0, 0, 83, 0, 0, 60, 0, 86, 0, 0, 87, 0, 0, 89, 0, 127, 0, 0, 61, 0, 58, 0, 76, 0, 0, 0, 0, 0]


## 模拟退火 ##

In [52]:
func = lambda x: inventory_cost(days, P1, D1, x)
x0 = our_Q_list # 起始点
lw = [0] * 60
up = [200] * 60

### 退火1次 ###

In [60]:
start = time.time()

x_solution = optimize.dual_annealing(func, list(zip(lw, up)), maxiter=1000, x0=x0)['x']
solution_Q_list = [int(i) for i in x_solution]
best_solution = inventory_cost_df(days, P1, D1, solution_Q_list)
best_solution

end = time.time()
run_time = end - start
print(run_time)

138.4317922592163


In [54]:
print([i for i in solution_Q_list])

[40, 0, 85, 0, 0, 87, 0, 0, 44, 0, 65, 0, 81, 0, 0, 50, 0, 135, 0, 0, 0, 54, 0, 53, 0, 117, 0, 0, 67, 0, 0, 133, 0, 0, 85, 0, 0, 62, 0, 82, 0, 0, 87, 0, 0, 89, 0, 127, 0, 0, 61, 0, 64, 0, 71, 0, 0, 0, 0, 0]


### 退火3次 ###

In [61]:
start = time.time()

best_cost = 99999999
for i in range(3):
    print(f'now round {i}:')    
    # 优化求解
    x_solution = optimize.dual_annealing(func, list(zip(lw, up)), maxiter=1000, x0=x0)['x']
    solution_Q_list = [int(i) for i in x_solution]
    # 记录更优结果
    if inventory_cost(days, P1, D1, solution_Q_list) < best_cost:
        best_cost = inventory_cost(days, P1, D1, solution_Q_list)
        best_solution = inventory_cost_df(days, P1, D1, solution_Q_list)
        x0 = solution_Q_list
        print(f'new record: {best_cost}')

best_solution

end = time.time()
run_time = end - start
print(run_time)

now round 0:
new record: 100069.095
now round 1:
now round 2:
393.3583309650421


In [56]:
print([i for i in solution_Q_list])

[40, 0, 85, 0, 0, 87, 0, 0, 38, 6, 63, 0, 81, 0, 0, 51, 0, 131, 0, 0, 0, 58, 0, 54, 0, 116, 0, 0, 68, 0, 0, 132, 0, 0, 83, 0, 0, 60, 0, 86, 0, 0, 88, 0, 0, 89, 0, 127, 0, 0, 61, 0, 77, 0, 58, 0, 0, 0, 0, 0]
